# Imports

In [ ]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# Data

The NYC taxi data is available to download [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

I used yellow taxi data for July, August, and September in 2018. All months together (Manhattan request) are approximately 22 million records.
Each record represents the one request which includes origin zone, destination zone, date, time, fare,...
you can find the description of each field [here](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)
However, this is the raw data and needs preprocessing. We turned the data into 8 timeslots a day for each zone, representing the number of passengers demand.

you can download the processed data [here]()

# Prepare training data

In [ ]:
data = pd.read_csv('july_august_september_taxi_data.csv') # read the data

In [ ]:
# we create a field for holiday data
# there are two holidays in July and September
data['holy']=0
data.at[data['date']=='07/04/2018','holy']=1
data.at[data['date']=='09/03/2018','holy']=1

In [ ]:
data=data.sort_values(['date','zone'], ascending=[True,True])

In [ ]:
def Normal(data,cont_cols):
    for col in cont_cols:
        mean = data[col].mean()
        std = data[col].std()
        data[col] = (data[col] - mean) / (1e-7 + std)

In [ ]:
# to predict the number of passengers for each time slot, 
# we used zone, day, holiday, and the number of passengers in the previous time slot
categorical_features = ['zone','day','holy']
cont_features = ['num_passenger']
output_feature = 'num_passenger'

In [ ]:
# we need to encode our features. for example, the name of the day turns from 1 to 7
label_encoders = {}
for cat_col in categorical_features:
        label_encoders[cat_col] = LabelEncoder()
        data[cat_col] = label_encoders[cat_col].fit_transform(data[cat_col])
Normal(data,cont_features)

In [ ]:
# every epoch receives data for a current week and predicts the coming week
date = np.unique(data['date'])
week=[]
for i in range(0,12):
    week.append(data[(data['date']>=date[7*i]) & (data['date']<date[7*i+7])][['zone','day','holy','num_passenger',]].values)

In [ ]:
# we set 11 weeks for train and the 1 week for test
x_train =[]
y_train =[]
for j in range(0,11):
    for i in range(0,int(len(week[0])/8)):
        x=week[j][i*8:i*8+8]
        y=week[j+1][i*8:i*8+8][:,3]
        x_train.append(x)
        y_train.append(y)
x_train=np.array(x_train)
y_train=np.array(y_train)

In [ ]:
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()

# Model

In [ ]:
#model
class MV_LSTM(nn.Module):

    def __init__(self, n_features, seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 32  # number of hidden states
        self.n_layers = 2  # number of LSTM layers (stacked)

        self.l_lstm = torch.nn.LSTM(input_size=n_features,
                                    hidden_size=self.n_hidden,
                                    num_layers=self.n_layers,
                                    batch_first=True)
        self.l_linear = torch.nn.Linear(self.n_hidden * self.seq_len, 8)

    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers, batch_size, self.n_hidden)
        cell_state = torch.zeros(self.n_layers, batch_size, self.n_hidden)
        self.hidden = (hidden_state, cell_state)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        lstm_out, self.hidden = self.l_lstm(x, self.hidden)
        x = lstm_out.contiguous().view(batch_size, -1)
        return self.l_linear(x)

In [ ]:
n_features = 4 # this is number of parallel inputs
n_timesteps = 8 # this is number of timeslots
mv_net = MV_LSTM(n_features,n_timesteps)
criterion = torch.nn.MSELoss() # loss
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)
train_episodes = 500
batch_size = len(x_train)

# train

In [ ]:
# we set the hyperparameters from the experiences were gained from training the other months of data
mv_net.train()
for t in range(train_episodes):
    for b in range(0, len(x_train), batch_size):
        inpt = x_train[b:b + batch_size, :, :]
        target = y_train[b:b + batch_size]

        x_batch = torch.tensor(inpt, dtype=torch.float32)
        y_batch = torch.tensor(target, dtype=torch.float32)

        mv_net.init_hidden(x_batch.size(0))
        output = mv_net(x_batch)
        loss = criterion(output, y_batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    if t%50 ==0:
        print('step : ', t, 'loss : ', loss.item())

# Test

In [ ]:
weektest=[]
weektest.append(data[(data['date']>=date[77]) & (data['date']<date[84])][['zone','day','holy','num_passenger']].values)
weektest.append(data[(data['date']>=date[84]) & (data['date']<=date[90])][['zone','day','holy','num_passenger']].values)
x_test =[]
y_test =[]
for i in range(0,int(len(weektest[0])/8)):
    x=weektest[0][i*8:i*8+8]
    y=week[1][i*8:i*8+8][:,3]
    x_test.append(x)
    y_test.append(y)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
mv_net.eval()
with torch.no_grad():
    x_test = torch.tensor(x_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)
    mv_net.init_hidden(x_test.size(0))
    output_t = mv_net(x_test)
    loss_test = criterion(output_t, y_test)
    print(loss_test.item())

# compare with SVR and LR

# SVR

In [ ]:
from sklearn.svm import SVR
modesvr_rbf = SVR(kernel='rbf', degree=2, gamma='auto',
tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200,
verbose=False, max_iter=-1)

modesvr_rbf.fit(x_train, y_train)
predicted_svr_rbf = modesvr_rbf.predict(x_test)
print("Model svr_rbf complete")
mse_svr_rbf =  mean_squared_error(predicted_svr_rbf, y_test)
print("Training data mean squared error: ",mse_svr_rbf)

In [ ]:
mean_absolute_error(predicted_svr_rbf, y_test)

# LR

In [ ]:
from sklearn import linear_model
model_lr = linear_model.Lasso(alpha = 0.1)

model_lr.fit(x_train, y_train)
predicted_LR = model_lr.predict(x_test)
print("Model lr complete")

mse_lr =  mean_squared_error(predicted_LR, y_test)
print("Training data mean squared error: ",mse_lr)

In [ ]:
mean_absolute_error(predicted_LR, y_test)